# Tests

In [26]:
import numpy as np
import networkx as nx
from dwdynamics import ComplexDynamicsProblem, Objective # Difference a. We are using ComplexDynamicsProblem
import dimod
import matplotlib.pyplot as plt
import json
import os
import minorminer
from dwave_networkx.generators.pegasus import (get_tuple_defragmentation_fn, fragmented_edges,
    pegasus_coordinates, pegasus_graph)
from dwave.system import DWaveSampler, EmbeddingComposite

In [27]:
def get_sampleset(topology, precision):
    # We simulate evolution under the Hamiltonian 1/2 pi * Sigma_y...
    SY = np.array([[0, -1.0j], [1.0j, 0.0]])
    PSI0 = np.array([1, 0], dtype=np.complex128)  # Difference b. Initial state can now be complex.

    # The problem object encapsulates all the data so you don't have to pass them around
    problem = ComplexDynamicsProblem(
        hamiltonian=0.5 * np.pi * SY,      # Evolution under hamiltonian 0.5 * np.pi * sigma_y
        initial_state = PSI0,              # Starting at initial state |0>,
        times=tuple(range(3)),             # With six time points 0,1,2
        num_bits_per_var=precision                # And two bits of precision per variable
    )
    qubo = problem.qubo(objective=Objective.norm)  # Other choice would be to use Objective.hessian
    num_int = qubo.num_interactions + 1

    while (num_int > qubo.num_interactions):
        num_int = qubo.num_interactions
        for key,value in qubo.quadratic.items():
            if value == 0:
                qubo.remove_interaction(key[0],key[1])

            
    assert qubo.num_variables == problem.hamiltonian.shape[0] * len(problem.times) * problem.num_bits_per_var * 2



    DW_TOKEN = os.getenv("DW_TOKEN")  # <- Place your token here, or use DW_TOKEN env variable, or configure D-Wave libraries to use config file


    # The graph fo the problem is pretty dense, we use automatic embedding to mitigate the incompatibility with actual machine
    dw_sampler = EmbeddingComposite(DWaveSampler(solver={'topology__type': topology}))


    dw_result = dw_sampler.sample(qubo, num_reads=1000, annealing_time=200)

    dw_result.first.energy
    return dw_result



In [31]:
for i in range(0,5):
    topology = 'pegasus'
    for precision in [2,3,4]:

        dw_result = get_sampleset(topology, precision)
        file_name = f"../data/{topology}_{precision}_{i}.json"
        with open(file_name,'w') as f:
            json.dump(dw_result.to_serializable(),f)

# Analyze

In [39]:
import os
import re
import pandas as pd
files = os.listdir('../data')

In [74]:
import dimod.sampleset

dfs = []

for topology in ['pegasus', 'zephyr']:
    for i in range(2,5):
        regex = re.compile(rf'{topology}_{i}_')

        precision_files = ([file for file in files if regex.search(file)])
        samplesets = [dimod.SampleSet.from_serializable(json.load(open(f'../data/{f}','r'))) for f in precision_files]
        samplesets = [s.to_pandas_dataframe()[['energy','num_occurrences']] for s in samplesets]
        s = pd.concat(samplesets)
        s['topology'] = topology
        s['precision'] = i
        dfs.append(s)


dfs_all = pd.concat(dfs)

In [81]:
dfs_grouped =dfs_all.groupby(['topology', 'precision','energy'])['num_occurrences'].sum()
dfs_grouped['pegasus',2]

energy
0.0       6
1.0      65
2.0     195
3.0     415
4.0     685
5.0     787
6.0     740
7.0     607
8.0     443
9.0     332
10.0    253
11.0    153
12.0    111
13.0     72
14.0     54
15.0     32
16.0     15
17.0      8
18.0      5
19.0      6
20.0      3
21.0      2
22.0      2
23.0      1
24.0      2
25.0      1
27.0      1
29.0      2
35.0      1
39.0      1
Name: num_occurrences, dtype: int64